# Non-negative matrix factorization using Autograd

In a [previous post](./nnmf-tensorflow.html), we had seen how to perfom non-negative matrix factorization (NNMF) using Tensorflow. In this post, we will look at performing NNMF using [Autograd](https://github.com/HIPS/autograd). Like Tensorflow, Autograd allows automatic gradient calculation.

### Customary imports

In [1]:
import autograd.numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Creating the matrix to be decomposed

In [2]:
A = np.array([[3, 4, 5, 2],
                   [4, 4, 3, 3],
                   [5, 5, 4, 3]], dtype=np.float32).T

### Masking one entry

In [3]:
A[0, 0] = np.NAN

In [4]:
A

array([[ nan,   4.,   5.],
       [  4.,   4.,   5.],
       [  5.,   3.,   4.],
       [  2.,   3.,   3.]], dtype=float32)

### Defining the cost function

In [13]:
def cost(param_list):
    W, H = param_list
    pred = np.dot(W, H)
    mask = ~np.isnan(A)
    return np.sqrt(((pred - A)[mask].flatten() ** 2).mean(axis=None))

### Decomposition params

In [14]:
rank = 2
learning_rate=0.01
n_steps = 10000

### Gradient of cost wrt params W and H

In [15]:
from autograd import grad, multigrad
grad_cost= grad(cost)

### Main gradient descent routine

In [16]:
shape = A.shape
H =  np.abs(np.random.randn(rank, shape[1]))
W =  np.abs(np.random.randn(shape[0], rank))
print "Iteration, Cost"
for i in range(n_steps):
    
    if i%1000==0:
        print "*"*20
        print i,",", cost([W, H])
    del_W, del_H = grad_cost([W, H])
    W =  W-del_W*learning_rate
    H =  H-del_H*learning_rate
    
    # Ensuring that W, H remain non-negative. This is also called projected gradient descent
    W[W<0] = 0
    H[H<0] = 0

Iteration, Cost
********************
0 , 2.96227720325
********************
1000 , 0.0982436473544
********************
2000 , 0.0869699720909
********************
3000 , 0.0866149349739
********************
4000 , 0.0865623391516
********************
5000 , 0.0865545044609
********************
6000 , 0.0865533304282
********************
7000 , 0.0865531539934
********************
8000 , 0.0865531274469
********************
9000 , 0.0865531234508


In [17]:
pd.DataFrame(W)

,0,1
0,0.997342,2.099647
1,1.412164,1.657989
2,0.257463,2.180860
3,1.286560,0.749794


In [18]:
pd.DataFrame(H)

,0,1,2
0,0.205837,1.550473,1.458757
1,2.260960,1.168605,1.688433


In [19]:
pred = np.dot(W, H)
pred_df = pd.DataFrame(pred).round()
pred_df

,0,1,2
0,5.0,4.0,5.0
1,4.0,4.0,5.0
2,5.0,3.0,4.0
3,2.0,3.0,3.0


In [20]:
pd.DataFrame(A)

,0,1,2
0,NaN,4.0,5.0
1,4.0,4.0,5.0
2,5.0,3.0,4.0
3,2.0,3.0,3.0
